# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링 (목표 user 1000명)

- user_profile_df : User의 아이디, 리뷰수, 팔로워, url을 저장함
- user_df : '아이디', '리뷰', '팔로워', '매장명', '카테고리', '주소', '리뷰 내용', '세부정보', '태그', '방문일자', '재방문횟수' 를 저장함


### 개요 

1. 매장 정보 크롤링 결과에서 업태구분 별로 리뷰가 많은 식당 n개를 골라 가져온다
2. 리뷰가 많은 식당 n개에서 각각 3명씩 ( 추천순 or 최신순 일지는 아직 미정) 

In [38]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.action_chains import ActionChains

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import os
from datetime import datetime

import pandas as pd
from selenium.webdriver.common.by import By

### 함수 

In [2]:
# 정규 표현식으로 이모티콘, 특수문자, 아스키코드 제거
def remove_special_characters(text):
    pattern = r'[^\w\s]|_'
    result = re.sub(pattern, '', text)
    return result

In [3]:
# User의 아이디, 리뷰수, 팔로워, url을 가져오는 함수 

def user_profile(url) :
    # 주소 이동

    global driver
    driver.get(url)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    user_element = {}

    # User ID 찾기
    user_id_element = soup.find('button', class_='wTaI4v _2kK3N- _2we3hB')
    user_element['아이디'] = user_id_element.text if user_id_element else None

    # User의 인기도 : 리뷰수, 팔로워 찾기 
    user_popularity = soup.find_all('button', class_='wTaI4v _15qVKh')
    
    for element in user_popularity:
        em_tag = element.find('em')
        if em_tag:
            key = element.text.replace(em_tag.text, '').strip()
            value = int(re.sub('[^0-9]', '', em_tag.text))  # 쉼표 제거 후 변환
            user_element[key] = value
    
    # User URL
    user_element['url'] = url

    print(user_element)

    return user_element

def change_date_format(날짜) :
    #  부분 추출 (년월일까지)
    date_part = ""
    for i in 날짜 :
        date_part += i
        date_part += ','
    date_part = date_part.split('\n')[1]

    # 일자, 요일, 방문일수 분류하기 
    date_part = date_part.split(',')

    # '21.1.3.'에서 숫자 부분 추출
    revisit_match = re.search(r'(\d+\.\d+\.\d+)\.', date_part[1])
    if revisit_match:
        revisit_str = revisit_match.group(1)
        # 방문일수 정수로 변환
        revisit = int(revisit_str.replace('.', ''))
    else:
        revisit = 0  # 또는 다른 기본값으로 설정

    # 일자 타입 변경하기 
    date = date_part[0]
    weekday = date[-3:]
    match = re.findall(r'(\d+)년 (\d+)월 (\d+)일', date) # 타입 바꾸기 
    
    if match:
        year, month, day = map(int, match[0])

        # 날짜 객체로 변환
        date_object = datetime(2000 + year, month, day) 
        formatted_date = date_object.strftime('%Y-%m-%d')
    else:
        formatted_date = ''  # 또는 다른 기본값으로 설정
    
    return formatted_date, weekday, revisit


### 1. 음식점 정보에서 카테고리 merge

In [5]:
res_category = pd.read_excel('./data/4_4_res_category_기준.xlsx')
res_category

,업태구분,카테고리
0,종합생활용품,없음
1,"지하철,전철",없음
2,백화점,없음
3,동물병원,없음
4,반려동물미용,없음
...,...,...
459,호두과자,간식
460,호텔,없음
461,화덕고깃간,고기
462,"화장품,향수",없음


In [6]:
매장_기본정보 = pd.read_excel('./data/2-1_naver_매장_기본정보_크롤링.xlsx')
매장_기본정보.drop('담당', axis=1, inplace=True)
매장_기본정보

,식당이름,업태구분,주소,메뉴,가격,방문자리뷰,블로그리뷰,검색어
0,하영호신촌설렁탕 역삼로점,"곰탕,설렁탕",서울 강남구 역삼로 215 1층,설렁탕,"11,000원",397,13,"역삼로 215, 신촌설렁탕"
1,사보텐 압구정본점,돈가스,서울 강남구 압구정로32길 32 1층,시그니처카츠(히레),"18,500원",555,109,압구정로32길 32 캘리스코 사보텐
2,피오렌티나,이탈리아음식,서울 강남구 논현로 841 제이비 미소 빌딩,디너 코스 1인,"70,000원",90,53,논현로 841 피오렌티나
3,압구정변강쇠떡볶이,떡볶이,서울 강남구 도산대로46길 21 한진로즈힐아파트 101동 상가117호,쌀떡볶이(4줄) 1인분,"4,500원",1583,526,도산대로46길 21 압구정변강쇠떡볶이
4,마포만두 역삼점,만두,서울 강남구 논현로 429,메뉴없음,가격없음,25,3,논현로 429 마포만두
...,...,...,...,...,...,...,...,...
8291,족발장인네 칼국수,한식,서울 강남구 강남대로114길 32 1층 101호,양지칼국수,"10,000원",33,32,"강남대로114길 32, 족발장인네 칼국수"
8292,사랑방,"전통,민속주점",서울 강남구 봉은사로1길 9,감자전,"18,000원",268,192,봉은사로1길 9 사랑방
8293,그때그집,"족발,보쌈",서울 강남구 선릉로129길 17,족발,"58,000원",295,147,선릉로129길 17 그때그집
8294,델리커리 삼성포스코점,카레,서울 강남구 테헤란로 440 포스코센터 지하 1층,3색 카레 (2인분),"35,000원",473,125,"테헤란로 440, 델리커리"


In [7]:
# 매장_기본정보와 res_category 데이터프레임을 조인하여 카테고리 정보를 가져와 새로운 컬럼으로 추가
res_group_df = pd.merge(매장_기본정보, res_category, on='업태구분', how='left')
res_group_df.drop('업태구분', axis=1, inplace=True)

# 방문자리뷰와 블로그리뷰를 더하여 리뷰합계 컬럼 생성
res_group_df['리뷰합계'] = res_group_df['방문자리뷰'] + res_group_df['블로그리뷰']
res_group_df

# 컬럼 순서 보기 좋기 변경
column_order =['검색어', '카테고리','식당이름', '주소', '메뉴', '가격', '방문자리뷰', '블로그리뷰',  '리뷰합계']
res_group_df = res_group_df[column_order]
res_group_df

,검색어,카테고리,식당이름,주소,메뉴,가격,방문자리뷰,블로그리뷰,리뷰합계
0,"역삼로 215, 신촌설렁탕",찜/탕/찌개,하영호신촌설렁탕 역삼로점,서울 강남구 역삼로 215 1층,설렁탕,"11,000원",397,13,410
1,압구정로32길 32 캘리스코 사보텐,일식,사보텐 압구정본점,서울 강남구 압구정로32길 32 1층,시그니처카츠(히레),"18,500원",555,109,664
2,논현로 841 피오렌티나,이탈리아,피오렌티나,서울 강남구 논현로 841 제이비 미소 빌딩,디너 코스 1인,"70,000원",90,53,143
3,도산대로46길 21 압구정변강쇠떡볶이,분식,압구정변강쇠떡볶이,서울 강남구 도산대로46길 21 한진로즈힐아파트 101동 상가117호,쌀떡볶이(4줄) 1인분,"4,500원",1583,526,2109
4,논현로 429 마포만두,한식,마포만두 역삼점,서울 강남구 논현로 429,메뉴없음,가격없음,25,3,28
...,...,...,...,...,...,...,...,...,...
8291,"강남대로114길 32, 족발장인네 칼국수",한식,족발장인네 칼국수,서울 강남구 강남대로114길 32 1층 101호,양지칼국수,"10,000원",33,32,65
8292,봉은사로1길 9 사랑방,술,사랑방,서울 강남구 봉은사로1길 9,감자전,"18,000원",268,192,460
8293,선릉로129길 17 그때그집,족발/보쌈,그때그집,서울 강남구 선릉로129길 17,족발,"58,000원",295,147,442
8294,"테헤란로 440, 델리커리",아시안,델리커리 삼성포스코점,서울 강남구 테헤란로 440 포스코센터 지하 1층,3색 카레 (2인분),"35,000원",473,125,598


### 2. 정보 확인

In [8]:
# 카테고리 별 숫자 세기
카테고리별_개수 = res_group_df['카테고리'].value_counts()

print(카테고리별_개수)

한식        1429
술         1042
카페         873
고기         844
일식         649
양식         376
중식         348
없음         321
전처리필요      319
분식         312
회해물        282
찜/탕/찌개     279
간식         187
치킨         151
아시안        146
이탈리아       105
피자         100
샐러드         96
족발/보쌈       88
햄버거         81
도시락         55
브런치         46
뷔페          39
죽           38
남미          24
퓨전음식        21
편의점         14
인도           8
스페인          8
야식           6
음료           6
푸드코트         3
Name: 카테고리, dtype: int64


### 3. 카테고리별 상위  10개 확인

In [10]:
# res_group_df를 카테고리별로 그룹화하고, 각 그룹에서 리뷰합계가 큰 상위 10개 선택
res_high = res_group_df.groupby('카테고리').apply(lambda x: x.nlargest(10, '리뷰합계')).reset_index(drop=True)

res_high

,검색어,카테고리,식당이름,주소,메뉴,가격,방문자리뷰,블로그리뷰,리뷰합계
0,"언주로168길 33, 런던베이글뮤지엄",간식,런던베이글뮤지엄 도산점,"서울 강남구 언주로168길 33 1, 2층",더블 베이컨 감자 샌드위치,"14,800원",4556,6860,11416
1,"영동대로 513, 에그슬럿",간식,에그슬럿 코엑스점,서울 강남구 영동대로 513 지하1층 101호,비프패티 페어팩스,"11,800원",7278,2008,9286
2,"밤고개로 99, 크리스피크림도넛",간식,크리스피크림 수서역사점,서울 강남구 밤고개로 99,오리지널 글레이즈드,"1,700원",5529,54,5583
3,"테헤란로 422, 던킨도너츠",간식,던킨 선릉역점,서울 강남구 테헤란로 422 KT타워 1층,베이컨에그 잉글리쉬머핀,"3,500원",4554,260,4814
4,"선릉로126길 14, 젠제로",간식,젠제로,서울 강남구 선릉로126길 14 예우빌딩 1층,소 포장 (3 flavors),"25,000원",3188,1313,4501
...,...,...,...,...,...,...,...,...,...
296,"논현로79길 62, 고래불",회해물,고래불,"서울 강남구 논현로79길 62 백악빌딩 1, 2층 고래불",독도꽃새우,변동,583,2073,2656
297,"언주로136길 10, 맛짱조개",회해물,맛짱조개,서울 강남구 언주로136길 10,조개찜 중,"55,000원",970,1339,2309
298,언주로136길 10 맛짱조개,회해물,맛짱조개,서울 강남구 언주로136길 10,조개찜 중,"55,000원",970,1337,2307
299,"강남대로156길 32, 골뱅이신사",회해물,골뱅이신사,서울 강남구 강남대로156길 32,생골뱅이탕 2인,"39,000원",592,1700,2292


### 4. 카테고리별 네이버 [리뷰] 탭에서 리뷰 상위 3개 쓴 user의 url 가져오기 

In [41]:
res_df = res_high.copy()
#res_df = res_df[0:2] # test용
res_df = res_df.reset_index(drop=True) # 인덱스 새로하기
res_df

,검색어,카테고리,식당이름,주소,메뉴,가격,방문자리뷰,블로그리뷰,리뷰합계
0,"언주로168길 33, 런던베이글뮤지엄",간식,런던베이글뮤지엄 도산점,"서울 강남구 언주로168길 33 1, 2층",더블 베이컨 감자 샌드위치,"14,800원",4556,6860,11416
1,"영동대로 513, 에그슬럿",간식,에그슬럿 코엑스점,서울 강남구 영동대로 513 지하1층 101호,비프패티 페어팩스,"11,800원",7278,2008,9286
2,"밤고개로 99, 크리스피크림도넛",간식,크리스피크림 수서역사점,서울 강남구 밤고개로 99,오리지널 글레이즈드,"1,700원",5529,54,5583
3,"테헤란로 422, 던킨도너츠",간식,던킨 선릉역점,서울 강남구 테헤란로 422 KT타워 1층,베이컨에그 잉글리쉬머핀,"3,500원",4554,260,4814
4,"선릉로126길 14, 젠제로",간식,젠제로,서울 강남구 선릉로126길 14 예우빌딩 1층,소 포장 (3 flavors),"25,000원",3188,1313,4501
...,...,...,...,...,...,...,...,...,...
296,"논현로79길 62, 고래불",회해물,고래불,"서울 강남구 논현로79길 62 백악빌딩 1, 2층 고래불",독도꽃새우,변동,583,2073,2656
297,"언주로136길 10, 맛짱조개",회해물,맛짱조개,서울 강남구 언주로136길 10,조개찜 중,"55,000원",970,1339,2309
298,언주로136길 10 맛짱조개,회해물,맛짱조개,서울 강남구 언주로136길 10,조개찜 중,"55,000원",970,1337,2307
299,"강남대로156길 32, 골뱅이신사",회해물,골뱅이신사,서울 강남구 강남대로156길 32,생골뱅이탕 2인,"39,000원",592,1700,2292


### 업태구분별로 헤비유저의 url 가져오기

In [43]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())

# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()
# driver = webdriver.Chrome()


user_url_info_df = pd.DataFrame()
search_name_list = []
user_url_list = []
user_name_list = []

max_review_count = 500 # 헤비유저를 결정하는 기준
max_user_url_count = 10 # 가져올 유저 url 숫자 

for i in range(0,len(res_df)):
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))
    time.sleep(3)  
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
    except :
        pass 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try :             
        # '리뷰' 탭의 href 속성 가져오기 
        review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'role': 'tab'})['href']
        review_url = 'https://pcmap.place.naver.com'+review_tab_href
        driver.get(review_url)
        time.sleep(5)  

        # 현재 페이지 URL 가져오기
        current_url = driver.current_url

        # 리뷰 url로 이동 
        modified_url = current_url.replace('/home', '/review/visitor')
        driver.get(modified_url)
    except :
        continue

    try:
        if driver.find_elements(By.ID, 'entryIframe'):
            entryIframe = driver.find_element(By.ID, 'entryIframe')
            driver.switch_to.frame(entryIframe)
            time.sleep(1)

        # url이 나타날때까지 스크롤 내리기 (컨디션을 좀 타서 안될때도 있음)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        
        # 리뷰에서 더보기를 몇번 클릭할 것인가?
        scroll_count = 0
        while scroll_count < 3 :
            try :
                # '더보기' 버튼을 찾기 (XPath를 사용하여 요소 선택)
                more_button = driver.find_element(By.CSS_SELECTOR, "a.fvwqf")
                actions = ActionChains(driver)
                actions.move_to_element(more_button).perform()
                time.sleep(1)
                more_button.click()
                scroll_count += 1
            except :
                break


        # html 긁어오기
        html_code = driver.page_source
        soup = BeautifulSoup(html_code, 'html.parser')
        # 리뷰들이 포함된 태그들을 선택
        reviews = soup.select('div.RKXdJ')

        count = 0

        for review in reviews:
            # 리뷰 갯수를 세서 헤비 업로더만 가져오기
            review_count = review.select_one('span.RNn6x').text
            review_count = int(review_count.replace('리뷰 ',''))
            # 네이버플레이스 리뷰 500개 이상인 user의 url만 가져옴 
            if review_count > max_review_count :
                # 리뷰 페이지의 href 가져오기
                user_url = review.select_one('a.j1rOp').get('href')
                user_name = review.select_one('span.P9EZi').text
            else : 
                continue

            ## 리스트 ㄱ
            search_name_list.append(name)
            user_url_list.append(user_url)
            user_name_list.append(user_name)

            # user의 url를 몇개나 가져올지 
            count += 1             
            if count >= max_user_url_count:  
                break  

    except Exception as e:
        print(f"에러 메시지: {str(e)}")

user_url_info_df = pd.DataFrame({'검색어' : search_name_list,
                                 '아이디' : user_name_list,
                             'url' : user_url_list
                             })
   
user_url_info_df

# 저장
user_url_info_df.to_excel('data/4_1_naver_user_크롤링_500개이상.xlsx', index=False)

